## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Resource/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Strezhevoy,60.7333,77.5889,10.40,67,0,8.95,RU,clear sky,2021-04-18 18:42:17
1,1,Ewa Beach,21.3156,-158.0072,75.99,64,1,8.05,US,clear sky,2021-04-18 18:39:59
2,2,Hobart,-42.8794,147.3294,55.40,58,100,11.50,AU,overcast clouds,2021-04-18 18:38:24
3,3,Hilo,19.7297,-155.0900,73.99,68,1,3.44,US,clear sky,2021-04-18 18:42:17
4,4,Rikitea,-23.1203,-134.9692,77.32,74,0,8.50,PF,clear sky,2021-04-18 18:42:17


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
1,1,Ewa Beach,21.3156,-158.0072,75.99,64,1,8.05,US,clear sky,2021-04-18 18:39:59
4,4,Rikitea,-23.1203,-134.9692,77.32,74,0,8.50,PF,clear sky,2021-04-18 18:42:17
6,6,Oistins,13.0667,-59.5333,86.00,51,40,24.16,BB,scattered clouds,2021-04-18 18:42:18
21,21,Thinadhoo,0.5333,72.9333,82.67,64,97,5.97,MV,overcast clouds,2021-04-18 18:42:24
32,32,Atuona,-9.8000,-139.0333,81.28,72,17,16.75,PF,few clouds,2021-04-18 18:42:27
34,34,Avarua,-21.2078,-159.7750,78.80,83,75,10.36,CK,light rain,2021-04-18 18:42:27
38,38,Puerto Ayora,-0.7393,-90.3518,82.40,74,75,9.22,EC,light rain,2021-04-18 18:40:57
39,39,Paracuru,-3.4100,-39.0306,81.00,77,77,21.99,BR,broken clouds,2021-04-18 18:42:29
49,49,Sao Joao Da Barra,-21.6403,-41.0511,86.00,51,40,3.44,BR,scattered clouds,2021-04-18 18:42:32
50,50,Mar Del Plata,-38.0023,-57.5575,78.01,68,75,9.22,AR,broken clouds,2021-04-18 18:38:24


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Country                200
Current Description    203
Date                   203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
1,1,Ewa Beach,21.3156,-158.0072,75.99,64,1,8.05,US,clear sky,2021-04-18 18:39:59
4,4,Rikitea,-23.1203,-134.9692,77.32,74,0,8.50,PF,clear sky,2021-04-18 18:42:17
6,6,Oistins,13.0667,-59.5333,86.00,51,40,24.16,BB,scattered clouds,2021-04-18 18:42:18
21,21,Thinadhoo,0.5333,72.9333,82.67,64,97,5.97,MV,overcast clouds,2021-04-18 18:42:24
32,32,Atuona,-9.8000,-139.0333,81.28,72,17,16.75,PF,few clouds,2021-04-18 18:42:27
...,...,...,...,...,...,...,...,...,...,...,...
663,663,Jacqueville,5.2052,-4.4146,86.00,74,20,10.36,CI,few clouds,2021-04-18 18:46:00
668,668,Bembereke,10.2283,2.6634,81.90,56,100,8.81,BJ,overcast clouds,2021-04-18 18:46:02
677,677,Placerville,38.7296,-120.7986,78.01,11,1,6.91,US,clear sky,2021-04-18 18:45:06
680,680,Sinnamary,5.3833,-52.9500,81.86,69,55,10.78,GF,broken clouds,2021-04-18 18:46:05


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ewa Beach,US,75.99,clear sky,21.3156,-158.0072,
4,Rikitea,PF,77.32,clear sky,-23.1203,-134.9692,
6,Oistins,BB,86.00,scattered clouds,13.0667,-59.5333,
21,Thinadhoo,MV,82.67,overcast clouds,0.5333,72.9333,
32,Atuona,PF,81.28,few clouds,-9.8000,-139.0333,
34,Avarua,CK,78.80,light rain,-21.2078,-159.7750,
38,Puerto Ayora,EC,82.40,light rain,-0.7393,-90.3518,
39,Paracuru,BR,81.00,broken clouds,-3.4100,-39.0306,
49,Sao Joao Da Barra,BR,86.00,scattered clouds,-21.6403,-41.0511,
50,Mar Del Plata,AR,78.01,broken clouds,-38.0023,-57.5575,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ewa Beach,US,75.99,clear sky,21.3156,-158.0072,Uncle Junior Robert
4,Rikitea,PF,77.32,clear sky,-23.1203,-134.9692,Pension Maro'i
6,Oistins,BB,86.00,scattered clouds,13.0667,-59.5333,Butterfly Beach Hotel
21,Thinadhoo,MV,82.67,overcast clouds,0.5333,72.9333,
32,Atuona,PF,81.28,few clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
663,Jacqueville,CI,86.00,few clouds,5.2052,-4.4146,Hotel Le Martin Pecheur
668,Bembereke,BJ,81.90,overcast clouds,10.2283,2.6634,Hotel Bio Guera
677,Placerville,US,78.01,clear sky,38.7296,-120.7986,Historic Cary House Hotel
680,Sinnamary,GF,81.86,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
print(locations)

         Lat       Lng
1    21.3156 -158.0072
4   -23.1203 -134.9692
6    13.0667  -59.5333
21    0.5333   72.9333
32   -9.8000 -139.0333
..       ...       ...
663   5.2052   -4.4146
668  10.2283    2.6634
677  38.7296 -120.7986
680   5.3833  -52.9500
681 -17.0000  145.4333

[200 rows x 2 columns]


In [13]:

len(hotel_info)

200

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))